In [152]:
import pandas as pd

In [153]:
data_imdb = pd.read_csv("input_datasets/fusion_without_release&nbrEntry.csv")
data_target = pd.read_csv("input_datasets/allocine.csv")




In [154]:
display(data_target.info())
display(data_imdb.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   actors          1680 non-null   object 
 1   director        1721 non-null   object 
 2   entrees_fr      1726 non-null   int64  
 3   entrees_usa     0 non-null      float64
 4   image_urls      1726 non-null   object 
 5   nationalite     1724 non-null   object 
 6   semaine_fr      1726 non-null   object 
 7   semaine_usa     0 non-null      float64
 8   studio          1647 non-null   object 
 9   timing          1726 non-null   int64  
 10  title           1726 non-null   object 
 11  titre_original  1699 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 161.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   titre              1697 non-null   object
 1   score              1697 non-null   object
 2   nbre_vote          1697 non-null   object
 3   genres             1697 non-null   object
 4   langue             1697 non-null   object
 5   pays               1697 non-null   object
 6   pegi               1697 non-null   object
 7   duree              1697 non-null   object
 8   annee              1697 non-null   object
 9   popularite_score   1697 non-null   object
 10  director           1697 non-null   object
 11  scenaristes        1697 non-null   object
 12  casting_principal  1697 non-null   object
 13  casting_complet    1697 non-null   object
 14  budget             1697 non-null   object
dtypes: object(15)
memory usage: 199.0+ KB


None

Fusion de deux datasets: 

In [155]:
import pandas as pd

# Renommer la colonne 'titre' du dataset 'data_imdb' pour correspondre à la colonne 'film_title' du dataset 'data_target'
# data_imdb.rename(columns={'titre': 'title'}, inplace=True)

data_imdb['titre'] = data_imdb['titre'].str.lower()
data_target['title'] = data_target['title'].str.lower()

# Fusionner les datasets en utilisant le titre du film comme clé de fusion
data = pd.merge(data_imdb, data_target, left_on='titre', right_on='title')

# Afficher le nouveau dataset fusionné
# display(data)


gestion probleme pegi et durée

In [156]:
# data.loc[data['duree'] == 0, 'duree'] = data['pegi']
# if (data['duree'].equals(0)):
#     data['duree'] = data['pegi']
# print(data['duree'])
data.loc[data['duree'] == "0", 'duree'] = data['pegi']

# # Initialiser le compteur à zéro
# nb_duree_zero = 0

# # Parcourir chaque valeur de la colonne 'duree'
# for duree in data['duree']:
#     if duree == "0":
#         nb_duree_zero += 1

# print("Nombre de fois où la durée est égale à 0 :", nb_duree_zero)


Gestion de Pegi :

In [157]:
data.loc[data['pegi'].str.match(r'^\d+'), 'pegi'] = None

# count_none = (data['pegi'] == "None").sum()
# print("Nombre de valeurs 'None' dans la colonne 'pegi' :", count_none)

Gestion durée:

In [158]:
def convert_timing(timing):
    try:
        if 'h' in timing and 'min' in timing:
            timing_cleaned = timing.replace('min', '') 
            hours, minutes = map(int, timing_cleaned.split('h '))
            total_minutes = hours * 60 + minutes
        elif 'h' in timing:
            hours = int(timing.replace('h', ''))
            total_minutes = hours * 60
        elif 'min' in timing:
            total_minutes = int(timing.replace('min', ''))
        else:
            total_minutes = 0
        return total_minutes
    except:
        return None


data['duree']= data['duree'].apply(convert_timing)

gestion du nbbr_vote

In [159]:
# for i, vote in enumerate(data['nbre_vote']):
    # if ',' in vote:
    #     data.at[i, 'nbre_vote'] = vote.replace(',', '')
    #     data.at[i, 'nbre_vote'] = data.at[i, 'nbre_vote'].replace(' k', '00')
    # else:
    #     data.at[i, 'nbre_vote'] = vote.replace(' k', '000')
def convert_vote(vote):
    try:
        if ',' in vote :
            vote_clean0 = vote.replace(',', '') 
            vote_clean = vote_clean0.replace("k", '00') 
        else:
            vote_clean = vote.replace("k", '000') 
        
        # vote_clean =  int(float(vote_clean))
        return vote_clean
    except:
        return None



data['nbre_vote']= data['nbre_vote'].apply(convert_vote)

In [160]:
display(data)

,titre,score,nbre_vote,genres,langue,pays,pegi,duree,annee,popularite_score,...,entrees_fr,entrees_usa,image_urls,nationalite,semaine_fr,semaine_usa,studio,timing,title,titre_original
0,heureux gagnants,"6,3",190,['Comédie'],['Français'],['France'],None,103,2024,['No Popularity Score'],...,136628,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,france,13/03/2024 au 20/03/2024,NaN,warner bros. france,0,heureux gagnants,pas de titre original
1,14 jours pour aller mieux,"5,6",50,['Comédie'],['Français'],['France'],None,90,2024,['No Popularity Score'],...,126678,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,france,06/03/2024 au 13/03/2024,NaN,wild bunch distribution,0,14 jours pour aller mieux,pas de titre original
2,la vie de ma mère,"6,9",79,"['Comédie', 'Drame']",['Français'],['France'],None,105,2023,['No Popularity Score'],...,58613,NaN,https://fr.web.img4.acsta.net/c_310_420/pictur...,france,06/03/2024 au 13/03/2024,NaN,kmbo,0,la vie de ma mère,pas de titre original
3,la zone d'intérêt,"7,5",73 000,"['Drame', 'Historique', 'Guerre']","['Allemand', 'Polonais', 'Yiddish']","['États-Unis', 'Royaume-Uni', 'Pologne']",Tous publics,105,2023,28,...,239108,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,u.s.a.,31 janvier au 7 février 2024,NaN,bac films,0,la zone d'intérêt,the zone of interest
4,il reste encore demain,"7,7",64 00,"['Comédie', 'Drame']",['Italien'],['Italie'],Tous publics,118,2023,552,...,145340,NaN,https://fr.web.img3.acsta.net/c_310_420/pictur...,italie,13/03/2024 au 20/03/2024,NaN,universal pictures international france,0,il reste encore demain,c'è ancora domani
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,ordinary angels,"7,5",35 00,['Drame'],['Anglais'],['États-Unis'],PG,118,2024,113,...,6500000,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,23/02/2024 au 26/02/2024,NaN,NaN,0,ordinary angels,pas de titre original
1337,out of darkness,"5,6",34 00,"['Horreur', 'Thriller']",['No Language'],['Royaume-Uni'],R,88,2022,634,...,0,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,grande-bretagne,09/02/2024 au 12/02/2024,NaN,NaN,0,out of darkness,pas de titre original
1338,i.s.s.,"5,2",65 00,"['Science-fiction', 'Thriller']","['Anglais', 'Russe']",['États-Unis'],R,95,2023,1 502,...,3064583,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,19/01/2024 au 22/01/2024,NaN,1 nomination,0,i.s.s.,pas de titre original
1339,the american society of magical negroes,"2,7",66 00,"['Comédie', 'Fantastique']",['Anglais'],"['États-Unis', 'Danemark']",PG-13,104,2024,45,...,1250000,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,u.s.a.,15/03/2024 au 18/03/2024,NaN,NaN,0,the american society of magical negroes,pas de titre original


formatage date:

In [161]:
mois_numeriques = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

# Fonction pour convertir le format de date pour allocine
def format_date_allocine(date_string):

    for m in mois_numeriques.keys():
        if m in date_string:
            splits = date_string.split()
            day = str(splits[0])
            month = str(mois_numeriques[splits[1]])
            year = str(splits[-1])
            date_format = day + "/" + month + "/" + year
            return date_format

    date_format = str(date_string.split(' au ')[0])
    return date_format

data['semaine_fr'] = data['semaine_fr'].apply(format_date_allocine)


In [162]:
cols_drop = ["timing", "semaine_usa", "image_urls", "entrees_usa"]

data = data.drop(cols_drop, axis=1)

#par manque de données:
cols_drop2 = ["budget"]
data = data.drop(cols_drop2, axis=1)

#par manque de données:
data_leaking = ["score", "nbre_vote", "popularite_score"]
data = data.drop(data_leaking, axis=1)


display(data)

,titre,genres,langue,pays,pegi,duree,annee,director_x,scenaristes,casting_principal,casting_complet,actors,director_y,entrees_fr,nationalite,semaine_fr,studio,title,titre_original
0,heureux gagnants,['Comédie'],['Français'],['France'],None,103,2024,Romain Choay,Romain Choay,"['Romain Choay', 'Maxime Govare', 'Scénario']","['Fabrice Eboué', 'Audrey Lamy', 'Anouk Grinbe...","Audrey Lamy,Fabrice Eboué,Anouk Grinberg","Maxime Govare,Romain Choay,Maxime Govare,Romai...",136628,france,13/03/2024,warner bros. france,heureux gagnants,pas de titre original
1,14 jours pour aller mieux,['Comédie'],['Français'],['France'],None,90,2024,Edouard Pluvieux,Edouard Pluvieux,"['Edouard Pluvieux', 'Scénario', 'Benjamin Dem...","['Maxime Gasteuil', 'Zabou Breitman', 'Romain ...","Maxime Gasteuil,Zabou Breitman,Romain Lancry","Edouard Pluvieux,Lionel Dutemple,Edouard Pluvieux",126678,france,06/03/2024,wild bunch distribution,14 jours pour aller mieux,pas de titre original
2,la vie de ma mère,"['Comédie', 'Drame']",['Français'],['France'],None,105,2023,Julien Carpentier,Julien Carpentier,"['Julien Carpentier', 'Scénario', 'Julien Carp...","['Agnès Jaoui', 'William Lebghil', 'Salif Ciss...","Agnès Jaoui,William Lebghil,Salif Cissé","Julien Carpentier,Julien Carpentier,Benjamin G...",58613,france,06/03/2024,kmbo,la vie de ma mère,pas de titre original
3,la zone d'intérêt,"['Drame', 'Historique', 'Guerre']","['Allemand', 'Polonais', 'Yiddish']","['États-Unis', 'Royaume-Uni', 'Pologne']",Tous publics,105,2023,Jonathan Glazer,Jonathan Glazer,"['Jonathan Glazer', 'Jonathan Glazer', 'Martin...","['Christian Friedel', 'Sandra Hüller', 'Johann...","Christian Friedel,Sandra Hüller,Johann Karthaus","Jonathan Glazer,Jonathan Glazer",239108,u.s.a.,31/01/2024,bac films,la zone d'intérêt,the zone of interest
4,il reste encore demain,"['Comédie', 'Drame']",['Italien'],['Italie'],Tous publics,118,2023,Paola Cortellesi,Paola Cortellesi,"['Paola Cortellesi', 'Furio Andreotti', 'Giuli...","['Paola Cortellesi', 'Valerio Mastandrea', 'Ro...","Paola Cortellesi,Valerio Mastandrea,Romana Mag...","Paola Cortellesi,Paola Cortellesi,Furio Andreotti",145340,italie,13/03/2024,universal pictures international france,il reste encore demain,c'è ancora domani
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,ordinary angels,['Drame'],['Anglais'],['États-Unis'],PG,118,2024,Jon Gunn,Jon Gunn,"['Jon Gunn', 'Meg Tilly', 'Kelly Fremon Craig']","['Hilary Swank', 'Alan Ritchson', 'Emily Mitch...","Hilary Swank,Alan Ritchson,Amy Acker","Jon Gunn,Kelly Fremon Craig",6500000,u.s.a.,23/02/2024,NaN,ordinary angels,pas de titre original
1337,out of darkness,"['Horreur', 'Thriller']",['No Language'],['Royaume-Uni'],R,88,2022,Andrew Cumming,Andrew Cumming,"['Andrew Cumming', 'Scénario', 'Ruth Greenberg']","['Iola Evans', 'Arno Lüning', 'Rosebud Melarke...","Chuku Modu,Kit Young,Safia Oakley-Green","Andrew Cumming,Ruth Greenberg",0,grande-bretagne,09/02/2024,NaN,out of darkness,pas de titre original
1338,i.s.s.,"['Science-fiction', 'Thriller']","['Anglais', 'Russe']",['États-Unis'],R,95,2023,Gabriela Cowperthwaite,Gabriela Cowperthwaite,"['Gabriela Cowperthwaite', 'Nick Shafir', 'Cas...","['Ariana DeBose', 'Chris Messina', 'John Galla...","Ariana DeBose,Chris Messina,Pilou Asbæk","Gabriela Cowperthwaite,Nick Shafir",3064583,u.s.a.,19/01/2024,1 nomination,i.s.s.,pas de titre original
1339,the american society of magical negroes,"['Comédie', 'Fantastique']",['Anglais'],"['États-Unis', 'Danemark']",PG-13,104,2024,Kobi Libii,Kobi Libii,"['Kobi Libii', 'Kobi Libii', 'Casting principal']","['Justice Smith', 'Zachary Barton', 'Anthony C...","Justice Smith,David Alan Grier,An-Li Bogan","Kobi Libii,Kobi Libii",1250000,u.s.a.,15/03/2024,NaN,the american society of magical negroes,pas de titre original


creation de continent en fonction du pays

In [163]:
# Modifier la feature pour ne garder que le premier pays
data['pays'] = data['pays'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())
data['pays'] = data['pays'].str.replace(']', '')
data['pays'] = data['pays'].str.replace('[', '')
data['pays'] = data['pays'].str.replace('"', '')
data['pays'] = data['pays'].str.replace("'", '')

# Obtenir une liste de tous les pays uniques
pays_uniques = data['pays'].unique()

# Afficher la liste des pays uniques
# print(pays_uniques)

In [164]:
# Créer un dictionnaire de correspondance entre les pays et les continents
correspondance_continents = {
    'France': 'Europe',
    'États-Unis': 'Amérique du Nord',
    'Italie': 'Europe',
    'Allemagne': 'Europe',
    'Royaume-Uni': 'Europe',
    'Irlande': 'Europe',
    'Canada': 'Amérique du Nord',
    'Chine': 'Asie',
    'Iran': 'Asie',
    'Corée du Sud': 'Asie',
    'Chili': 'Amérique du Sud',
    'Pologne': 'Europe',
    'Jordanie': 'Asie',
    'Belgique': 'Europe',
    'Japon': 'Asie',
    'Malaisie': 'Asie',
    'Argentine': 'Amérique du Sud',
    'Suède': 'Europe',
    'Espagne': 'Europe',
    'Finlande': 'Europe',
    'Danemark': 'Europe',
    'Norvège': 'Europe',
    'Israël': 'Asie',
    'Turquie': 'Asie',
    'Guinée-Bissau': 'Afrique',
    'Mexique': 'Amérique du Nord',
    'Portugal': 'Europe',
    'Mongolie': 'Asie',
    'Russie': 'Europe',
    'Yémen': 'Asie',
    'Australie': 'Océanie',
    'Inde': 'Asie',
    'Algérie': 'Afrique',
    'Suisse': 'Europe',
    'Ukraine': 'Europe',
    'Maroc': 'Afrique',
    'Autriche': 'Europe',
    'Indonésie': 'Asie',
    'Cameroun': 'Afrique',
    'Brésil': 'Amérique du Sud',
    'Vietnam': 'Asie',
    'Soudan': 'Afrique',
    'Taïwan': 'Asie',
    'Bhoutan': 'Asie',
    'Lituanie': 'Europe',
    'République tchèque': 'Europe',
    'Islande': 'Europe',
    'Serbie': 'Europe',
    'No Country': 'Inconnu',
    'Hong Kong': 'Asie',
    'Pakistan': 'Asie',
    'Égypte': 'Afrique',
    'Bosnie-Herzégovine': 'Europe',
    'Tunisie': 'Afrique',
    'Hongrie': 'Europe',
    'Colombie': 'Amérique du Sud',
    'Guatemala': 'Amérique du Nord',
    'Roumanie': 'Europe',
    'Sénégal': 'Afrique',
    'Slovaquie': 'Europe',
    'Pérou': 'Amérique du Sud',
    'Pays-Bas': 'Europe',
    'Haïti': 'Amérique du Nord',
    'North Macedonia': 'Europe',
    'Liban': 'Asie',
    'Philippines': 'Asie',
    'Kosovo': 'Europe',
    'Territoires palestiniens occupés': 'Asie',
    'Syrie': 'Asie',
    'Lettonie': 'Europe',
    'Grèce': 'Europe',
    'Afrique du Sud': 'Afrique',
    'Croatie': 'Europe',
    'Kazakhstan': 'Asie',
    'Rwanda': 'Afrique',
    'Allemagne de lOuest': 'Europe',
    'Antilles néerlandaises': 'Amérique du Nord',
    'Géorgie': 'Europe',
    'Luxembourg': 'Europe',
    'Cambodge': 'Asie',
    'Thaïlande': 'Asie',
    'Bolivie': 'Amérique du Sud',
    'Costa Rica': 'Amérique du Nord',
    'Bulgarie': 'Europe',
    'Laos': 'Asie'
}

# Ajouter une nouvelle colonne 'continent' en utilisant la correspondance
data['continent'] = data['pays'].map(correspondance_continents)
# print(data['pays'])
# Afficher les premières lignes pour vérification
# print(data.head())

# Enregistrer le dataset modifié avec la nouvelle colonne continent
data.to_csv("votre_fichier_modifié.csv", index=False)

In [165]:
# Trouver les valeurs de la feature 'duree' qui contiennent des lettres
duree_contenant_lettres = data[pd.to_numeric(data['duree'], errors='coerce').isna()]

# Afficher les lignes avec les valeurs de 'duree' contenant des lettres
print(duree_contenant_lettres)

Empty DataFrame
Columns: [titre, genres, langue, pays, pegi, duree, annee, director_x, scenaristes, casting_principal, casting_complet, actors, director_y, entrees_fr, nationalite, semaine_fr, studio, title, titre_original, continent]
Index: []


Gestion des genre pour n'en garder qu'un

In [167]:
data['genres'] = data['genres'].str.replace(']', '')
data['genres'] = data['genres'].str.replace('[', '')
data['genres'] = data['genres'].str.replace('"', '')
data['genres'] = data['genres'].str.replace("'", '')

data['genres'] = data['genres'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())


In [168]:
#exportation du dataset netoye
display(data)

data.to_csv("datasets/dataset_cleaned.csv", index=None)

data.to_pickle("datasets/dataset.pkl")

,titre,genres,langue,pays,pegi,duree,annee,director_x,scenaristes,casting_principal,casting_complet,actors,director_y,entrees_fr,nationalite,semaine_fr,studio,title,titre_original,continent
0,heureux gagnants,Comédie,['Français'],France,None,103,2024,Romain Choay,Romain Choay,"['Romain Choay', 'Maxime Govare', 'Scénario']","['Fabrice Eboué', 'Audrey Lamy', 'Anouk Grinbe...","Audrey Lamy,Fabrice Eboué,Anouk Grinberg","Maxime Govare,Romain Choay,Maxime Govare,Romai...",136628,france,13/03/2024,warner bros. france,heureux gagnants,pas de titre original,Europe
1,14 jours pour aller mieux,Comédie,['Français'],France,None,90,2024,Edouard Pluvieux,Edouard Pluvieux,"['Edouard Pluvieux', 'Scénario', 'Benjamin Dem...","['Maxime Gasteuil', 'Zabou Breitman', 'Romain ...","Maxime Gasteuil,Zabou Breitman,Romain Lancry","Edouard Pluvieux,Lionel Dutemple,Edouard Pluvieux",126678,france,06/03/2024,wild bunch distribution,14 jours pour aller mieux,pas de titre original,Europe
2,la vie de ma mère,Comédie,['Français'],France,None,105,2023,Julien Carpentier,Julien Carpentier,"['Julien Carpentier', 'Scénario', 'Julien Carp...","['Agnès Jaoui', 'William Lebghil', 'Salif Ciss...","Agnès Jaoui,William Lebghil,Salif Cissé","Julien Carpentier,Julien Carpentier,Benjamin G...",58613,france,06/03/2024,kmbo,la vie de ma mère,pas de titre original,Europe
3,il reste encore demain,Comédie,['Italien'],Italie,Tous publics,118,2023,Paola Cortellesi,Paola Cortellesi,"['Paola Cortellesi', 'Furio Andreotti', 'Giuli...","['Paola Cortellesi', 'Valerio Mastandrea', 'Ro...","Paola Cortellesi,Valerio Mastandrea,Romana Mag...","Paola Cortellesi,Paola Cortellesi,Furio Andreotti",145340,italie,13/03/2024,universal pictures international france,il reste encore demain,c'è ancora domani,Europe
4,bolero,Comédie,['Anglais'],États-Unis,None,105,1984,John Derek,John Derek,"['John Derek', 'John Derek', 'Casting principal']","['Bo Derek', 'George Kennedy', 'Andrea Occhipi...","Raphaël Personnaz,Doria Tillier,Jeanne Balibar","Anne Fontaine,Anne Fontaine,Claire Barré",177724,france,06/03/2024,snd,bolero,pas de titre original,Amérique du Nord
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,love lies bleeding,Action,['Anglais'],Royaume-Uni,R,104,2024,Rose Glass,Rose Glass,"['Rose Glass', 'Rose Glass', 'Weronika Tofilska']","['Anna Baryshnikov', 'Kristen Stewart', 'Dave ...","Kristen Stewart,Katy O'Brian,Anna Baryshnikov","Rose Glass,Rose Glass,Weronika Tofilska",2485650,grande-bretagne,15/03/2024,1 nomination,love lies bleeding,pas de titre original,Europe
1067,ordinary angels,Drame,['Anglais'],États-Unis,PG,118,2024,Jon Gunn,Jon Gunn,"['Jon Gunn', 'Meg Tilly', 'Kelly Fremon Craig']","['Hilary Swank', 'Alan Ritchson', 'Emily Mitch...","Hilary Swank,Alan Ritchson,Amy Acker","Jon Gunn,Kelly Fremon Craig",6500000,u.s.a.,23/02/2024,NaN,ordinary angels,pas de titre original,Amérique du Nord
1068,out of darkness,Horreur,['No Language'],Royaume-Uni,R,88,2022,Andrew Cumming,Andrew Cumming,"['Andrew Cumming', 'Scénario', 'Ruth Greenberg']","['Iola Evans', 'Arno Lüning', 'Rosebud Melarke...","Chuku Modu,Kit Young,Safia Oakley-Green","Andrew Cumming,Ruth Greenberg",0,grande-bretagne,09/02/2024,NaN,out of darkness,pas de titre original,Europe
1069,the american society of magical negroes,Comédie,['Anglais'],États-Unis,PG-13,104,2024,Kobi Libii,Kobi Libii,"['Kobi Libii', 'Kobi Libii', 'Casting principal']","['Justice Smith', 'Zachary Barton', 'Anthony C...","Justice Smith,David Alan Grier,An-Li Bogan","Kobi Libii,Kobi Libii",1250000,u.s.a.,15/03/2024,NaN,the american society of magical negroes,pas de titre original,Amérique du Nord
